# Pfizer 2023 Hail Workshop
## 04: Advanced Hail Functionality

This notebook is a grab bag of more advanced Hail functionality.

### Approximate CDF

Normally computing quantiles or the median requires sorting an entire dataset. Hail uses a sophisticated data structure to get provably good approximations of all quantiles without sorting the data, providing buckets, or using unbounded memory.

In [1]:
import os
# Give Hail a bunch of RAM
os.environ['PYSPARK_SUBMIT_ARGS'] = '--executor-memory 16G --driver-memory 16G pyspark-shell'

In [2]:
import hail as hl
hl.plot.output_notebook()

Loading BokehJS ...

In [3]:
mt = hl.read_matrix_table('resources/1kg.mt')
mt = hl.variant_qc(mt)
cdf = mt.aggregate_rows(hl.agg.approx_cdf(mt.variant_qc.call_rate))
cdf

Initializing Hail with default parameters...


2023-01-11 14:45:40.378 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2023-01-11 14:45:42.185 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2023-01-11 14:45:42.186 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


Running on Apache Spark version 3.1.3
SparkUI available at http://148.168.60.57:4042
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.107-2387bb00ceee
LOGGING: writing to /Users/dking/projects/pfizer-2023-01-11/04/hail-20230111-1445-0.2.107-2387bb00ceee.log


Struct(values=[0.5633802816901409, 0.6161971830985915, 0.6795774647887324, 0.6971830985915493, 0.8028169014084507, 0.8556338028169014, 0.8661971830985915, 0.9049295774647887, 0.9225352112676056, 0.9330985915492958, 0.9401408450704225, 0.9436619718309859, 0.9471830985915493, 0.9507042253521126, 0.954225352112676, 0.9577464788732394, 0.9612676056338029, 0.9647887323943662, 0.9683098591549296, 0.971830985915493, 0.9753521126760564, 0.9788732394366197, 0.9823943661971831, 0.9859154929577465, 0.9894366197183099, 0.9929577464788732, 0.9964788732394366, 1.0], ranks=[0, 1, 2, 3, 5, 9, 25, 57, 58, 122, 186, 250, 314, 378, 379, 444, 572, 700, 828, 956, 1212, 1597, 2109, 2621, 3645, 5053, 6909, 9149, 10879], _compaction_counts=[165, 61, 23, 8, 3, 1, 0, 0])

In [4]:
import bokeh.plotting as bp

def plot_cdf(cdf, title):
    values = cdf['values']
    values = values + [values[-1]]
    ranks = cdf['ranks']
    ranks = [x / ranks[-1] for x in ranks]

    p = bp.figure(title=title, plot_width=400, plot_height=400)
    p.step(x=[0] + values, y=[0] + ranks, line_width=2, line_color='black')

    hl.plot.show(p)
    
plot_cdf(cdf, 'Approximate CDF of Call Rate')

In [5]:
mt = hl.read_matrix_table('resources/1kg.mt')
mt = hl.variant_qc(mt)
cdf = mt.aggregate_rows(hl.agg.approx_cdf(mt.variant_qc.AF[0]))
plot_cdf(cdf, 'Approximate CDF of Reference AF')

You can also ask directly for the median:

In [6]:
mt.aggregate_rows(hl.agg.approx_median(mt.variant_qc.AF[0]))

0.7307692307692307

### PCA on Unusual Values

Flexible, general-purpose methods enable analysts to explore data sets with novel statistics.

In [7]:
mt = hl.read_matrix_table('resources/1kg.mt')
mt = mt.filter_rows(hl.agg.any(hl.is_missing(mt.GT)))
mt = mt.annotate_entries(
    is_missing = hl.is_missing(mt.GT)
)
mt = mt.annotate_rows(
    is_missing_stats = hl.agg.stats(mt.is_missing)
)
mt = mt.annotate_entries(
    normed_is_missing = (mt.is_missing - mt.is_missing_stats.mean) / mt.is_missing_stats.stdev
)
_, scores, _ = hl.pca(mt.normed_is_missing, k=2)
hl.plot.show(hl.plot.scatter(scores.scores[0], scores.scores[1]))

2023-01-11 14:47:26.961 Hail: INFO: pca: running PCA with 2 components...4) / 4]
2023-01-11 14:47:32.481 Hail: INFO: wrote table with 0 rows in 0 partitions to /tmp/persist_tablet5ConpVaEV
    Total size: 6.26 KiB
    * Rows: 0.00 B
    * Globals: 6.26 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A


### LD Prune

In [9]:
?hl.ld_prune

In [8]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')
print(f'Before pruning we have: {mt.count_rows()}')
pruned_variants = hl.ld_prune(mt.GT)
pruned_variants.write('output/pruned_variants.ht', overwrite=True)
pruned_variants = hl.read_table('output/pruned_variants.ht')

mt = mt.filter_rows(hl.is_defined(pruned_variants[mt.row_key]))
print(f'After pruning we have: {mt.count_rows()}')

Before pruning we have: 19151


2023-01-11 14:48:09.127 Hail: INFO: ld_prune: running local pruning stage with max queue size of 61681 variants
2023-01-11 14:48:19.298 Hail: INFO: wrote table with 2389 rows in 100 partitions to /tmp/persist_tabletUZWOBnbAf
    Total size: 95.93 KiB
    * Rows: 95.92 KiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  126 rows (4.49 KiB)
2023-01-11 14:48:24.414 Hail: INFO: wrote table with 2389 rows in 100 partitions to /tmp/1GbEd3sczwrKGSu4mV68JG
2023-01-11 14:48:38.503 Hail: INFO: Wrote all 2 blocks of 2389 x 4151 matrix with block size 4096.
2023-01-11 14:48:48.126 Hail: INFO: wrote table with 1027 rows in 1 partition to /tmp/ipcajvgdPpB5J74dF3IrLO
    Total size: 49.82 KiB
    * Rows: 15.05 KiB
    * Globals: 34.77 KiB
    * Smallest partition: 1027 rows (15.05 KiB)
    * Largest partition:  1027 rows (15.05 KiB)
2023-01-11 14:48:54.138 Hail: INFO: wrote table with 1920 rows in 100 partitions to /tmp/persist_tablehddJ5XwGbw
2023-01-11 14

After pruning we have: 1920


### Kinship Estimators

Hail supports a number of different kinship estimators.

Getting PC Relate to produce good-looking results is tricky! Here we see what happens when you don't quality control the variants well enough.

In [10]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')

pc_kin = hl.pc_relate(mt.GT, 0.1, k=4, statistics='kin20', min_kinship=0.1)
pc_kin.write('output/pc_kin.ht', overwrite=True)
pc_kin = hl.read_table('output/pc_kin.ht')

hl.plot.show(
    hl.plot.scatter(
        pc_kin.kin,
        pc_kin.ibd0,
        width=400,
        height=400,
        size=3
    )
)

2023-01-11 14:49:56.437 Hail: INFO: hwe_normalize: found 19151 variants after filtering out monomorphic sites.
2023-01-11 14:50:00.494 Hail: INFO: pca: running PCA with 4 components... / 100]
2023-01-11 14:50:14.420 Hail: INFO: wrote table with 0 rows in 0 partitions to /tmp/persist_tablerWSLMmzZmL
    Total size: 155.63 KiB
    * Rows: 0.00 B
    * Globals: 155.63 KiB
    * Smallest partition: N/A
    * Largest partition:  N/A
2023-01-11 14:50:23.619 Hail: INFO: Wrote all 10 blocks of 19151 x 4151 matrix with block size 4096.
2023-01-11 14:50:24.968 Hail: INFO: wrote matrix with 5 rows and 19151 columns as 5 blocks of size 4096 to /tmp/pcrelate-write-read-7w2g7F0nv3EpAzKdGPmnfo.bm
2023-01-11 14:50:25.467 Hail: INFO: wrote matrix with 19151 rows and 4151 columns as 10 blocks of size 4096 to /tmp/pcrelate-write-read-6ojsB8LWNAOEUtbs29nrcK.bm
2023-01-11 14:50:27.113 Hail: INFO: wrote matrix with 19151 rows and 4151 columns as 10 blocks of size 4096 to /tmp/pcrelate-write-read-su6UN1FqnMY

In [11]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')

king_kin = hl.king(mt.GT)
king_kin = king_kin.filter_entries(king_kin.phi > 0.1).entries()
king_kin.write('output/king_kin.ht', overwrite=True)
king_kin = hl.read_table('output/king_kin.ht')

hl.plot.show(
    hl.plot.histogram(
        king_kin.phi
    )
)

2023-01-11 14:51:36.891 Hail: INFO: Wrote all 10 blocks of 19151 x 4151 matrix with block size 4096.
2023-01-11 14:51:42.811 Hail: INFO: Wrote all 10 blocks of 19151 x 4151 matrix with block size 4096.
2023-01-11 14:51:49.255 Hail: INFO: Wrote all 10 blocks of 19151 x 4151 matrix with block size 4096.
2023-01-11 14:51:54.697 Hail: INFO: Wrote all 10 blocks of 19151 x 4151 matrix with block size 4096.
2023-01-11 14:52:02.657 Hail: INFO: wrote matrix with 4151 rows and 4151 columns as 4 blocks of size 4096 to /tmp/icEy71iOgKWyKp1oEnTS1n
2023-01-11 14:52:10.180 Hail: INFO: wrote matrix with 4151 rows and 4151 columns as 4 blocks of size 4096 to /tmp/d3IhDPkISALQjZkBvHRzbj
2023-01-11 14:52:17.277 Hail: INFO: wrote matrix with 4151 rows and 4151 columns as 4 blocks of size 4096 to /tmp/DBKFebgFovXoyisCBglL5x
2023-01-11 14:52:18.300 Hail: INFO: wrote matrix with 4151 rows and 4151 columns as 4 blocks of size 4096 to /tmp/bbhFh5OIsvuDH2qTG6UNGz
2023-01-11 14:52:18.993 Hail: INFO: wrote matrix

In [19]:
king_kin.filter(king_kin.phi < 0.45).show()

,,
s_1,s,phi
str,str,float64
"""CHMI_CHMI3_WGS2""","""HG00114""",1.30e-01
"""CHMI_CHMI3_WGS2""","""HG00183""",1.06e-01
"""CHMI_CHMI3_WGS2""","""HG00243""",1.01e-01
"""CHMI_CHMI3_WGS2""","""HG00246""",1.01e-01
"""CHMI_CHMI3_WGS2""","""HG00282""",1.03e-01
"""CHMI_CHMI3_WGS2""","""HG00308""",1.16e-01
"""CHMI_CHMI3_WGS2""","""HG00311""",1.04e-01
"""CHMI_CHMI3_WGS2""","""HG00351""",1.25e-01


Hail also supports identity-by-descent calculation but it's currently broken for the new Apple M1 chips because it uses some fast native code that hasn't been compiled for M1 yet. Expect a fix soon!

### Polygenic Score Calculation

In this section, I import a height polygenic score from the [PGS Catalog](https://www.pgscatalog.org/score/PGS000297/), and use it to calculate the polygenic score in our toy dataset. Our toy dataset does not have enough shared variants with the score to produce useful estimates, but the code below could be effectively applied to a larger, quality-controlled dataset.

In [13]:
ht = hl.import_table('resources/height-polygenic-score.txt', comment='#', impute=True)
ht = ht.key_by(
    locus = hl.locus(hl.str(ht.chr_name), ht.chr_position)
)
ht.write('output/height-polygenic-score.ht', overwrite=True)

2023-01-11 14:53:25.098 Hail: INFO: wrote table with 3291 rows in 1 partition to /tmp/persist_tableivSmIMEc2i
2023-01-11 14:53:26.114 Hail: INFO: Reading table to impute column types
2023-01-11 14:53:27.469 Hail: INFO: Finished type imputation
  Loading field 'rsID' as type str (imputed)
  Loading field 'chr_name' as type int32 (imputed)
  Loading field 'chr_position' as type int32 (imputed)
  Loading field 'effect_allele' as type str (imputed)
  Loading field 'other_allele' as type str (imputed)
  Loading field 'effect_weight' as type float64 (imputed)
  Loading field 'variant_description' as type str (imputed)
2023-01-11 14:53:28.156 Hail: INFO: Coerced sorted dataset
2023-01-11 14:53:28.995 Hail: INFO: wrote table with 3290 rows in 1 partition to output/height-polygenic-score.ht


In [14]:
ht = hl.read_table('output/height-polygenic-score.ht')

In [15]:
ht.show()

,,,,,,,
rsID,chr_name,chr_position,effect_allele,other_allele,effect_weight,variant_description,locus
str,int32,int32,str,str,float64,str,locus<GRCh37>
"""rs385039""",1,2077409,"""G""","""A""",2.05e-02,"""lead SNPs""",1:2077409
"""rs12743493""",1,2224836,"""A""","""G""",1.39e-02,"""lead SNPs""",1:2224836
"""rs2843146""",1,2265969,"""T""","""G""",5.37e-03,"""secondary""",1:2265969
"""rs4648832""",1,2286127,"""G""","""C""",9.18e-03,"""secondary""",1:2286127
"""rs16823193""",1,2847985,"""T""","""A""",1.11e-02,"""lead SNPs""",1:2847985
"""rs6704012""",1,3038530,"""C""","""T""",8.42e-03,"""secondary""",1:3038530
"""rs10909940""",1,3316756,"""A""","""G""",1.13e-02,"""lead SNPs""",1:3316756
"""rs17404435""",1,3766286,"""G""","""T""",9.08e-03,"""lead SNPs""",1:3766286


In [16]:
mt = hl.read_matrix_table('resources/1kg.mt')
mt = hl.variant_qc(mt)
mt = mt.annotate_rows(score=ht[mt.locus])

mt = mt.annotate_rows(is_flipped = (
    hl.case()
    .when(mt.score.effect_allele == mt.alleles[0], True)
    .when(mt.score.effect_allele == mt.alleles[1], False)
    .or_missing()
))
mt = mt.annotate_rows(
    mean_gt=2 * hl.if_else(mt.is_flipped, mt.variant_qc.AF[0], mt.variant_qc.AF[1])
)
mt = mt.annotate_entries(
    n_effect_alleles = hl.if_else(
        mt.is_flipped,
        2 - mt.GT.n_alt_alleles(),
        mt.GT.n_alt_alleles()
    )
)
mt = mt.annotate_cols(
    prs = hl.agg.sum(mt.score.effect_weight * hl.coalesce(mt.n_effect_alleles, mt.mean_gt)),
    n_useful_variants = hl.agg.sum(hl.is_defined(mt.score.effect_weight))
)
mt.cols().show()

,,
s,prs,n_useful_variants
str,float64,int64
"""HG00096""",2.97e-01,13
"""HG00099""",3.27e-01,13
"""HG00105""",2.27e-01,13
"""HG00118""",2.83e-01,13
"""HG00129""",3.16e-01,13
"""HG00148""",2.59e-01,13
"""HG00177""",3.41e-01,13
"""HG00182""",4.03e-01,13


### LD Score

Hail also has utilities for simulating phenotypes, calculating LD Scores, and running LD Score regression.

In [20]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')
mt = hl.experimental.ldscsim.simulate_phenotypes(mt, mt.GT, h2=0.5)
mt.y.show()

calculating phenotype


,
s,y
str,float64
"""CHMI_CHMI3_WGS2""",-4.61e-01
"""LP6005441-DNA_F08""",-1.40e+00
"""HGDP00843""",1.59e-01
"""HGDP00392""",3.12e-01
"""LP6005441-DNA_H03""",-8.34e-01
"""HGDP00544""",5.66e-01
"""HGDP01053""",5.25e-03
"""HGDP00191""",-1.26e-01


In [21]:
betas = hl.linear_regression_rows(y=mt.y, x=mt.GT.n_alt_alleles(), covariates=[1.0])

2023-01-11 14:56:13.312 Hail: INFO: linear_regression_rows: running on 4151 samples for 1 response variable y,
    with input variable x, and 1 additional covariate...
2023-01-11 14:56:20.075 Hail: INFO: wrote table with 19147 rows in 100 partitions to /tmp/persist_table4gVjdpdl74
    Total size: 1.22 MiB
    * Rows: 1.22 MiB
    * Globals: 11.00 B
    * Smallest partition: 0 rows (21.00 B)
    * Largest partition:  517 rows (32.20 KiB)


In [22]:
betas.show()

,,,,,,,,
locus,alleles,n,sum_x,y_transpose_x,beta,standard_error,t_stat,p_value
locus<GRCh38>,array<str>,int32,float64,float64,float64,float64,float64,float64
chr1:14464,"[""A"",""T""]",4151,9.99e+02,2.75e+01,5.15e-02,3.52e-02,1.46e+00,1.43e-01
chr1:16298,"[""C"",""T""]",4151,4.45e+03,-1.15e+02,-1.11e-01,4.04e-02,-2.75e+00,6.04e-03
chr1:16378,"[""T"",""C""]",4151,4.04e+03,-7.14e+01,-8.81e-02,5.77e-02,-1.53e+00,1.27e-01
chr1:16487,"[""T"",""C""]",4151,7.06e+02,-5.18e+01,-7.45e-02,4.01e-02,-1.86e+00,6.30e-02
chr1:16495,"[""G"",""C""]",4151,4.08e+03,-5.53e+01,-1.22e-01,1.18e-01,-1.03e+00,3.02e-01
chr1:16841,"[""G"",""T""]",4151,5.01e+02,-7.69e+00,-4.44e-03,4.68e-02,-9.48e-02,9.24e-01
chr1:17020,"[""G"",""A""]",4151,1.76e+03,-2.28e+01,-2.58e-03,3.11e-02,-8.30e-02,9.34e-01
chr1:17147,"[""G"",""A""]",4151,4.73e+02,-3.86e+00,3.97e-03,4.84e-02,8.20e-02,9.35e-01


In [28]:
?hl.experimental.ld_score

In [23]:
ht_scores = hl.experimental.ld_score(entry_expr=mt.GT.n_alt_alleles(),
                                     locus_expr=mt.locus,
                                     radius=1e6)


betas = betas.annotate(z_score = betas.beta / betas.standard_error)
betas = betas.annotate(chi_sq_statistic = betas.z_score ** 2)

ht = mt.rows()

ht_results = hl.experimental.ld_score_regression(
    weight_expr=ht_scores[ht.locus].univariate,
    ld_score_expr=ht_scores[ht.locus].univariate,
    chi_sq_exprs=betas[ht.key].chi_sq_statistic,
    n_samples_exprs=betas[ht.key].n
)

2023-01-11 14:56:36.015 Hail: INFO: Wrote all 10 blocks of 19147 x 4151 matrix with block size 4096.
2023-01-11 14:57:01.615 Hail: INFO: wrote matrix with 19147 rows and 19147 columns as 15 blocks of size 4096 to /tmp/xSPnI5xVVcn6XeJAiRzWim
2023-01-11 14:57:02.961 Hail: INFO: wrote matrix with 19147 rows and 1 column as 5 blocks of size 4096 to /tmp/UNmxNYTaoSg6YI5sKo7HFZ
2023-01-11 14:57:03.654 Hail: INFO: merging 5 files totalling 343.7K...
2023-01-11 14:57:03.683 Hail: INFO: while writing:
    /tmp/4fGMYYqi3jxceDnVUkwocF
  merge time: 27.676ms
2023-01-11 14:57:04.704 Hail: INFO: wrote table with 19147 rows in 1 partition to /tmp/persist_tablekM4gtc1o6v
2023-01-11 14:57:05.116 Hail: INFO: Reading table to impute column types
2023-01-11 14:57:05.619 Hail: INFO: Finished type imputation
  Loading field 'f0' as type float64 (imputed)
2023-01-11 14:57:13.436 Hail: INFO: Coerced sorted dataset======>(99 + 1) / 100]
2023-01-11 14:57:14.284 Hail: INFO: Coerced sorted dataset
2023-01-11 14:5

In [25]:
ht_results.write('output/ldsr.ht', overwrite=True)
ldsr = hl.read_table('output/ldsr.ht')
ldsr.show()

2023-01-11 14:58:29.997 Hail: INFO: wrote table with 1 row in 1 partition to output/ldsr.ht


+-----------+-------------+--------------------+--------------------------+
| phenotype | mean_chi_sq | intercept.estimate | intercept.standard_error |
+-----------+-------------+--------------------+--------------------------+
|     int32 |     float64 |            float64 |                  float64 |
+-----------+-------------+--------------------+--------------------------+
|         0 |    1.08e+01 |           1.53e+00 |                 1.89e-01 |
+-----------+-------------+--------------------+--------------------------+

+---------------------------+---------------------------------+
| snp_heritability.estimate | snp_heritability.standard_error |
+---------------------------+---------------------------------+
|                   float64 |                         float64 |
+---------------------------+---------------------------------+
|                  3.45e-01 |                        2.58e-02 |
+---------------------------+---------------------------------+

### Annotation Database

The Hail team maintains a database of common variant annotations in Google Cloud Storage and S3. These commands will only work when executed inside a cluster with access to Google Cloud Storage or S3.

A full list of available annotations can be found [in the Hail docs](https://hail.is/docs/0.2/annotation_database_ui.html).

In [ ]:
mt = hl.read_matrix_table('resources/1kg.mt')

db = hl.experimental.DB(region='us', cloud='aws')
mt = db.annotate_rows_db(
    mt,
    'CADD', 'GTEx_eQTL_Adipose_Subcutaneous_all_snp_gene_associations', 'gnomad_ld_scores_afr'
)
mt.rows().show()

### VEP

Hail also supports VEP annotation. This requires a specially configured cluster.

In [ ]:
mt = hl.read_matrix_table('resources/1kg.mt')
mt = hl.vep(mt)
mt.vep.show()

In [26]:
mt = hl.read_matrix_table('resources/qced-hgdp-1kg.mt')
mt.vep.show()

+---------------+------------+-------------------+-------------------+
| locus         | alleles    | vep.assembly_name | vep.allele_string |
+---------------+------------+-------------------+-------------------+
| locus<GRCh38> | array<str> | str               | str               |
+---------------+------------+-------------------+-------------------+
| chr1:14464    | ["A","T"]  | "GRCh38"          | "A/T"             |
| chr1:16298    | ["C","T"]  | "GRCh38"          | "C/T"             |
| chr1:16378    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:16487    | ["T","C"]  | "GRCh38"          | "T/C"             |
| chr1:16495    | ["G","C"]  | "GRCh38"          | "G/C"             |
| chr1:16841    | ["G","T"]  | "GRCh38"          | "G/T"             |
| chr1:17020    | ["G","A"]  | "GRCh38"          | "G/A"             |
| chr1:17147    | ["G","A"]  | "GRCh38"          | "G/A"             |
| chr1:17385    | ["G","A"]  | "GRCh38"          | "G/A"             |
| chr1:17407    | ["G","A"]  | "GRCh38"          | "G/A"             |
+---------------+------------+-------------------+-------------------+

+---------------+-------------+---------+--------+---------------------------+
| vep.ancestral | vep.context | vep.end | vep.id | vep.input                 |
+---------------+-------------+---------+--------+---------------------------+
| str           | str         |   int32 | str    | str                       |
+---------------+-------------+---------+--------+---------------------------+
| NA            | NA          |   14464 | "."    | "chr1	14464	.	A	T	.	.	GT" |
| NA            | NA          |   16298 | "."    | "chr1	16298	.	C	T	.	.	GT" |
| NA            | NA          |   16378 | "."    | "chr1	16378	.	T	C	.	.	GT" |
| NA            | NA          |   16487 | "."    | "chr1	16487	.	T	C	.	.	GT" |
| NA            | NA          |   16495 | "."    | "chr1	16495	.	G	C	.	.	GT" |
| NA            | NA          |   16841 | "."    | "chr1	16841	.	G	T	.	.	GT" |
| NA            | NA          |   17020 | "."    | "chr1	17020	.	G	A	.	.	GT" |
| NA            | NA          |   17147 | "."    | "chr1	17147	.	G	A	.	.	GT" |
| NA            | NA          |   17385 | "."    | "chr1	17385	.	G	A	.	.	GT" |
| NA            | NA          |   17407 | "."    | "chr1	17407	.	G	A	.	.	GT" |
+---------------+-------------+---------+--------+---------------------------+

+------------------------------------------------------------------------------+
| vep.intergenic_consequences                                                  |
+------------------------------------------------------------------------------+
| array<struct{allele_num: int32, consequence_terms: array<str>, impact: st... |
+------------------------------------------------------------------------------+
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
| NA                                                                           |
+------------------------------------------------------------------------------+

+--------------------------------------+
| vep.most_severe_consequence          |
+--------------------------------------+
| str                                  |
+--------------------------------------+
| "non_coding_transcript_exon_variant" |
| "intron_variant"                     |
| "intron_vari

In [31]:
mt = mt.annotate_rows(
    interesting_cnsq = mt.vep.transcript_consequences.find(lambda x: x.consequence_terms.contains("stop_gained"))
)
mt = mt.filter_rows(hl.is_defined(mt.interesting_cnsq))
mt.interesting_cnsq.show(n=30)

+---------------+---------------------------------+
| locus         | alleles                         |
+---------------+---------------------------------+
| locus<GRCh38> | array<str>                      |
+---------------+---------------------------------+
| chr1:1318875  | ["G","A"]                       |
| chr1:1320653  | ["C","T"]                       |
| chr1:2430625  | ["C","CGTGGGTGAGTGAGGCCCTGGCT"] |
| chr1:2559459  | ["C","A"]                       |
| chr1:3783866  | ["G","T"]                       |
+---------------+---------------------------------+

+-----------------------------+------------------------------+
| interesting_cnsq.allele_num | interesting_cnsq.amino_acids |
+-----------------------------+------------------------------+
|                       int32 | str                          |
+-----------------------------+------------------------------+
|                           1 | "Q/*"                        |
|                           1 | "W/*"                        |
|                           1 | "-/VGE*GPGX"                 |
|                           1 | "C/*"                        |
|                           1 | "Y/*"                        |
+-----------------------------+------------------------------+

+-------------------------+---------------------------+
| interesting_cnsq.appris | interesting_cnsq.biotype  |
+-------------------------+---------------------------+
| str                     | str                       |
+-------------------------+---------------------------+
| NA                      | "protein_coding"          |
| NA                      | "nonsense_mediated_decay" |
| NA                      | "protein_coding"          |
| NA                      | "protein_coding"          |
| NA                      | "nonsense_mediated_decay" |
+-------------------------+---------------------------+

+----------------------------+-----------------------+
| interesting_cnsq.canonical | interesting_cnsq.ccds |
+----------------------------+-----------------------+
|                      int32 | str                   |
+----------------------------+-----------------------+
|                         NA | NA                    |
|                         NA | NA                    |
|                         NA | NA                    |
|                         NA | NA                    |
|                         NA | NA                    |
+----------------------------+-----------------------+

+-----------------------------+---------------------------+
| interesting_cnsq.cdna_start | interesting_cnsq.cdna_end |
+-----------------------------+---------------------------+
|                       int32 |                     int32 |
+-----------------------------+---------------------------+
|                         908 |                       908 |
|                         151 |                       151 |
|                         346 |                       347 |
|                         326 |                       326 |
|                          98 |                        98 |
+-----------------------------+---------------------------+

+--------------------------+----------------------------+
| interesting_cnsq.cds_end | interesting_cnsq.cds_start |
+--------------------------+----------------------------+
|                    int32 |                      int32 |
+--------------------------+----------------------------+
|                      850 |                        850 |
|                      153 |                        153 |
|                      112 |                        111 |
|                      291 |                        291 |
|                       99 |                         99 |
+--------------------------+----------------------------+

+----------------------------+------------------------------------------+
| interesting_cnsq.codons    | interesting_cnsq.consequence_terms       |
+----------------------------+--------------------------------